In [ ]:
# pip install folium (in Anaconda Environment)
!pip install folium

In [ ]:
from IPython.display import HTML
import folium
import json


In [ ]:
folium.__version__

In [ ]:
map = folium.Map(location=[13.794619, 100.323492])
map

## Visualizing Spatial Data with Folium


```
folium.Map()
```



This class method will always be the first thing that you execute when working with Folium. The purpose of this function is to generate the default map object that will be rendered by your notebook, and the object that we will be building on top of for our visualizations.

In [ ]:
import folium

center = [13.794619, 100.323492]

def generateBaseMap(default_location=center, default_zoom_start=6):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

There are several parameters within this class function that I usually use, which are:



1.   ``location``, this parameter basically define the default location that will be shown by the map as the center location.

2.  ``zoom_start``, which defines the default magnification level of the map

3.  ``control_scale``, enables/disables the map scale for a given zoom level. This is something that sometimes can be handy to have for us to get an idea of the scale of the geographical area that we are looking at.

There are also many other parameters that can be set within this class method.

In [ ]:
base_map = generateBaseMap()
base_map

In [ ]:
folium.Marker(location =[13.794619, 100.323492]).add_to(base_map)
base_map

### Getting Data

In [ ]:
import sqlite3
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')


In [ ]:
conn = sqlite3.connect('data/thailand_cities.db')
df_cities = pd.read_sql_query('SELECT * FROM cities', conn)

In [ ]:
df_cities.head(2)

## `Marker()`

In [ ]:
# create empty map zoomed in on Salaya
base_map = generateBaseMap([13.794619, 100.323492], 5)
 
# add a marker for every record in the map
for city in df_cities.iterrows():
    folium.Marker(location = [city[1]['Latitude'],city[1]['Longitude']]).add_to(base_map)
    
base_map

## `MarkerCluster()`

In [ ]:
from folium.plugins import MarkerCluster

# create empty map zoomed in on Salaya
base_map = generateBaseMap([13.794619, 100.323492], 5)

# create marker cluster
mc = MarkerCluster()
     
# add a marker for every record in the filtered data, use a clustered view
for city in df_cities.iterrows():
    mc.add_child(folium.Marker(location = [city[1]['Latitude'],city[1]['Longitude']]))

mc.add_to(base_map)
base_map

## ``HeatMap()``

In [ ]:
df_pm = pd.read_sql_query('SELECT * FROM pm_by_city', conn)
df_pm.head()

In [ ]:
df_cities.head()

In [ ]:
pm_city = pd.read_sql_query('SELECT c.City, Latitude, Longitude, Year, Month, Day, [UTC Hour], [PM2.5] FROM pm_by_city pm JOIN cities c ON pm.city = c.City', conn)
pm_city.head()

In [ ]:
pm_city.tail()

In [ ]:
pm_city[['Latitude', 'Longitude', 'PM2.5']].isnull().sum()

In [ ]:
pm_city[['Latitude', 'Longitude', 'PM2.5']].dtypes

In [ ]:
for c in pm_city.columns:
    pm_city[c] = pd.to_numeric(pm_city[c], errors='ignore')

In [ ]:
pm_city[['Latitude', 'Longitude', 'PM2.5']].dtypes

In [ ]:
data = pm_city[['City', 'Latitude', 'Longitude', 'PM2.5']].groupby(['City', 'Latitude', 'Longitude']).max().reset_index()
data.sort_values('PM2.5', ascending=False).head(10)

In [ ]:
from folium import plugins
from folium.plugins import HeatMap

base_map = generateBaseMap()
#HeatMap(data[['Latitude', 'Longitude', 'PM2.5']], radius=20, max_zoom=13).add_to(base_map)

HeatMap(data[['Latitude', 'Longitude', 'PM2.5']], radius = 20,gradient={.6: 'lime', .98: 'orange', 1: 'red'}).add_to(base_map)
base_map

## ``HeatMapWithTime()``

We can also animate our heat maps to change the data being shown on it based on certain dimension(s) (i.e. hour, month) using a class method called ``HeatMapWithTime()``

In [ ]:
pm_hour_list = []

for hour in pm_city['UTC Hour'].sort_values().unique():
    pm_hour_list.append(pm_city.loc[pm_city['UTC Hour'] == hour, ['Latitude', 'Longitude', 'UTC Hour', 'PM2.5']].groupby(['Latitude', 'Longitude', 'UTC Hour']).max().reset_index().values.tolist())
    
pm_hour_list[5]

In [ ]:
from folium.plugins import HeatMapWithTime

base_map = generateBaseMap(default_zoom_start=6)
HeatMapWithTime(pm_hour_list, radius=10, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8).add_to(base_map)
base_map

In [ ]:
pm_month_list = []

for hour in pm_city['Month'].sort_values().unique():
    pm_month_list.append(pm_city.loc[pm_city['Month'] == hour, ['Latitude', 'Longitude', 'Month', 'PM2.5']].groupby(['Latitude', 'Longitude', 'Month']).mean().reset_index().values.tolist())

    
pm_month_list

In [ ]:
from folium.plugins import HeatMapWithTime

base_map = generateBaseMap(default_zoom_start=6)
HeatMapWithTime(pm_month_list, radius=10, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8).add_to(base_map)
base_map

In [ ]:
# definition of the boundaries in the map
with open('data/thailand.json') as json_file:  
    thai_geo = json.load(json_file)

In [ ]:
# Initialize the map:
#m = folium.Map([13.794619, 100.323492], zoom_start=6)

base_map = generateBaseMap(default_zoom_start=6)
folium.GeoJson(thai_geo).add_to(base_map)

base_map

In [ ]:
pm_city.head()


data = pm_city[['City', 'PM2.5']].groupby(['City']).max().reset_index()
data.sort_values('PM2.5', ascending=False).head(10)

data.head()

## ``Choropleth()``

A choropleth map is a thematic map in which areas are shaded in proportion to the measurement of the numeric value, such as population density, income, PM2.5 per city. 

In [ ]:
from branca.colormap import linear

colormap = linear.YlGn_09.scale(
    data['PM2.5'].min(),
    data['PM2.5'].max())

print(colormap(5.0))

colormap

In [ ]:
data_dict = data.set_index('City')['PM2.5']

data_dict.head()

In [ ]:
base_map = generateBaseMap(default_zoom_start=6)


folium.Choropleth(
    geo_data=thai_geo,
    name='choropleth',
    data=data_dict,
    columns=['City', 'PM2.5'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(base_map)


folium.LayerControl().add_to(base_map)

base_map